# Main contributors

targets - 위 문제는 mask에 따라 경계를 찾아서 instance segmetation 문제로 풀 수 있습니다서

loss function - 픽셀 불균형이 결과에 영향을 미치지 않는 방식으로 cross entropy와 soft dice loss를 결합

매우 깊은 방식의 인코더-디코더 아키텍처 (이런 최신 방식은 좋지만 다른 segmentation 문제를 야기합니다.(SpaceNet, Inria 등))

watershed, morphological features, Gradient boosted tree와 second-level model과 결합된 까다로운 후처리<sup>postprocessing</sup>

태스크별 data augmentation

이러한 기술을 결합하여 하나의 모델로도 public LB에서 1위를 달성할 수 있었습니다.



Main contributions

    targets - we predict touching borders along with the masks to solve the problem as instance segmentation
    loss function - that combines crossentropy and soft dice loss in such a way that pixel imbalance doesn't affect the results
    very deep encoder-decoder architectures that also achieve state-of-the-art results in other binary segmentation problems (SpaceNet, Inria and others)
    tricky postprocessing that combines watershed, morphological features and second-level model with Gradient Boosted Trees
    task specific data augmentations

Combined these tricks allowed to achieve 1st place on public LB even with a single model.

##**U-Net vs Mask-RCNN?**

이전에는 segmentation challenges(Carvana, Urban 3D, Spacenet, Konica Minolta)에서 실제로 경쟁했고 뭐가 더 좋은지는 당연했습니다. -**Unet on steroids**

U-Net vs Mask-RCNN?

We had experience in segmentation challenges before (Carvana, Urban 3D, Spacenet, Konica Minolta) where we actually competed against each other and therefore the choice was clear - UNet on steroids. 

## Target masks

먼저 가장 간단한 접근법을 시도했고 binary masks(즉, 항상 핵 사이에 간격이 있도록 수정된 GT 마스크)를 위해 watershed line을 추가했다.

이러한 접근 방식을 통해 0.500 정도의 public LB를 얻을 수 있습니다.

분명히 우승할만한 점수는 아닙니다.


그런 다음 contour<sup>등고선</sup>가포함된 두번째 채널을 추가했는데, 여기서 contour의 폭은 핵 크기에 따라 달라집니다.

이 mask+단순 watershed post processing을 하면 올바른 분석 방향을 알 수 있습니다.

잘못 예측된 결과를 살펴보면, network는 모호하지 않은 장소의 contour는 쉽게 예측하며 핵을 분리하기 위해 등고선이 핵을 분리하기 위해 실제로 필요한 곳에서는 예측에 실패하는 것이 분명했다. 

그래서 세포 사이의 경계선에서만 예측하기로 결정했다. 

이를 통해 single network 결과 + watershed postprocessing으로 550 이상의 결과를 쉽게 얻을 수 있었습니다.

만약 한 채널과 다른 채널의 경계에 full mask를 갖고 있다면, *가끔 watershed를 위한 seed가 충분히 좋지 않습니다

더 좋은 접근 방법은 nuclei mask를 바꾸고 경계에 있는 픽셀을 비우는 거죠.

이를 통해 sigmoid 대신에 softmax를 target activation으로 사용할 수 있다. 

이는 핵을 더 잘 분리하지만 IoU에 대한 높은 임계값 때문에 실제론 MaP를 감소시킨다.

전체 mask에 대해 학습된 추가 네트워크가 이러한 문제를 해결하며 전 후처리 단계에서 두 결과를 통합했습니다.


Target masks

First we tried the simplest approach and added watershed line for binary masks(i.e. modified GT masks to always have gaps between nuclei). This approach gave us around ~500 on Public LB. Clearly this is not enough to win.

Then we added second channel with contours, where width of contours depended on the nucleus size. These masks + simple watershed postprocessing gave us ~525, not a breakthrough but a hint to the right direction.

Looking at the mistakes it was clear that networks easily predict contours in non ambiguous places and fail miserably in places where we actually need contours to separate the nuclei. Hence we decided to predict only the borders between the cells. This easily gave us 550+ on the LB with a single network results + watershed postprocessing.

Still if we have a full mask in one channel and a border in another, sometimes the seeds for watershed are not good enough. A better approach is to change nuclei masks and make pixels empty on the borders. This also allows to use softmax as the target activation instead of sigmoid. This better separates nuclei but actually decreases MaP because of high thresholds for IoU. We solved this issue with additional networks trained on full masks and combined the results in the postprocessing step.

대상에 대한 최종 접근:

*   sigmoid activation의 네트워크에 대한 2 channel masks(mask-border, border)나 softmax activation의 네트워크에 대한 3 channel mask(mask-border, 1-mask-border)

*  2 channels full mask (mask, border)

c43e356beedae15fec60ae3f8b06ea8e9036081951deb7e44f481b15b3acfc37 의 경우 다음과 같이 보입니다. 
![alt text](https://uc051918c8d6fd97e1fc1e795362.dl.dropboxusercontent.com/cd/0/get/ATfdQHjeepIuuKlrX2aFTXdWme14xjCXf2NSX5bgwHdw4-0knc1k-6KbzL-teDKEsl57QICIZEYDppU0EoraG5ECN65Q5HMHv5_3nsj5uZPDaFB-J3d1FvD-mCyMhZzOpN8K2JQ-_4BpDMggopayH1k_hyT95WE9nrf39DGIHtyO68SpisA98DfhHOyVDuUik_8/file?dl=1)

후처리 후에 최종 이미지

![alt text](https://uc75660d1caa4d8cf94c59580455.dl.dropboxusercontent.com/cd/0/get/ATceFZenA6qZ5Lx7HHQf3wCvDTRmGBRtDYv-ur9yCmaN4Zc4-lPO08la_keppLBmSBRUB4lClBfIE4_akwW43qSGRXbDBePPcqpFsTtWR2jY21N6YaoXJ58-ZGmUXZIpKuktEBq6je5OOoIJlI-TV7ft5_oFHgbO-h8KxrphRt7OZ8AveSvZr_2LzUgmjFafpak/file?dl=1 )

Final approach for targets:

    2 channels masks for networks with sigmoid activation i.e. (mask - border, border) or 3 channels masks for networks with softmax activation i.e. (mask - border, border , 1 - mask - border)

    2 channels full masks i.e. (mask, border)

For c43e356beedae15fec60ae3f8b06ea8e9036081951deb7e44f481b15b3acfc37 it looks like this 

## Augmentations

수백개의 훈련 이미지가 있었기 대문에, 모델이 오버피팅되는 것을 방지해야 하면, 다소 일반화 하기 위해 구체적인 augmentation이 필요했다. 우리는 많은 augmentation 이미지를 사용했다 (어쩌면 너무 heavy할 수도 있다)


    Color to Gray
    Inverting - we should not have used it, some images were not predicted correctly on stage2 because of this augmentation
    Remapping grayscale images to random color images
    Blur, Median Blur, Motion Blur
    contrast and brightness
    random scale, rotates and flips
    Heavy geometric transformations: Elastic Transform, Perspective Transform, Piecewise Affine transforms, pincushion distortion
    Random HSV
    Channel shuffle - I guess this one was very important due to the nature of the data
    Nucleus copying on images. That created a lot of overlapping nuclei. It seemed to help networks to learn better borders for overlapping nuclei.


*   Clahe, Sharpen, Emboss
*   Gaussian Noise
*   Color to Gray
*   Inverting- 이건 사용하지 말아야합니다. 이 augmentation 때문에 몇몇 이미지가 stage2에서 제대로 예측되지 않습니다.
*    Remapping grayscale images to random color images
*    Blur, Median Blur, Motion Blur
*    contrast and brightness
*   random scale, rotates and flips
*    Heavy geometric transformations: Elastic Transform, Perspective Transform, Piecewise Affine transforms, pincushion distortion
*    Random HSV
*   Channel shuffle - 데이터 특성상 매우 중요했던 것 같습니다.
*   Nucleus copying on images. 많은 overlapping nuclei를 만들어냅니다. 핵에 대해 더 나은 경계를 배우도록 돕는 것 같습니다. 











Augmentations

As there were just some hundreds of training images, we needed to come up with specific augmentations to prevent our models from overfitting and make them more or less generalizable. We used a lot of heavy augmentations (maybe too heavy)

    Clahe, Sharpen, Emboss
    Gaussian Noise
    Color to Gray
    Inverting - we should not have used it, some images were not predicted correctly on stage2 because of this augmentation
    Remapping grayscale images to random color images
    Blur, Median Blur, Motion Blur
    contrast and brightness
    random scale, rotates and flips
    Heavy geometric transformations: Elastic Transform, Perspective Transform, Piecewise Affine transforms, pincushion distortion
    Random HSV
    Channel shuffle - I guess this one was very important due to the nature of the data
    Nucleus copying on images. That created a lot of overlapping nuclei. It seemed to help networks to learn better borders for overlapping nuclei.


## Network architectures

ImageNet에서 pretrain된 인코더의 인코더 디코더 U-net을 사용했습니다. 

놀랍게도, VGG16과 같은 간단한 인코더는 이번 competition에서 썩 좋지는 않았습니다. 

이런 모델들은 핵처럼 보이는 조직같이 어려운 image에서는 예측에 실패했지만 특히 59b35151ddc7a5dd7ab7f171b142cfe40ee 같은 경우에서는 잘 예측했다. 

이러한 실험 후에 더 깊게 들어가기로 결정했다!! 그 결과 이 대회에서 최고 성능을 발휘하는 encoder는 DPN-152, InceptionResnetV2, Resnet101이었다. 

Network architectures

We used UNet like encoder-decoder architectures with encoders pretrained on ImageNet.

Surprisingly, simple encoders like VGG16 did not work in this competition at all. They failed on the hard cases when the tissue looked like nucleus but it was not, especially on color images like 59b35151d4a7a5ffdd7ab7f171b142db8cfe40beeee67277fac6adca4d042c4

After these experiments we decided that we have to go deeper!!! As a result - top performing encoders in this competition were : DPN-92, Resnet-152, InceptionResnetV2, Resnet101. 

## 2단계 모델/ 후처리<sup>Post Processing</sup>

예측된 핵 candidate<sup>후보</sup>에 대해 LightGBM 모델을 학습시켰다. 각각의 기본 candidate들은 분할을 위한 가장 낮은 임계값<sup>threshold</sup>으로 선택되었고 높은 임계값과 침식<sup>erosion</sup>이 거의 없게 분리되도록 노력했습니다.

solidity, circularity, convexity, area, neightbors median area, count 등과 같은 candidate에 대한 기본 형태학적 특징을 거의 사용하지 않았다. 예측 대상 -  ground truth iou (0, iou<0.5일 때)

그런 다음 예측된 iou에 따라 각 candidate에 최적의 분리 임계값이 선택됩니다.

낮은 예측 IoU의 candidate는 제거된다. (iou<0.3과 2개의 제출에 대해 iou<0.2 - 이러한 경우 augmentation에도 불구하고 오버피팅이 있었기 때문에 OOF 예측을 이용해서 임계값을 찾기가 어려웠다.   )


2nd Level model / Postprocessing

LightGBM models trained on predicted nucleus candidates. Each base candidate selected with lowest threshold for separation and tried to be separated with few higher thresholds and erosion. Used few basic morphological features about candidate like solidity, circularity, convexity, area, neighbors median area, count, etc.. Target for prediction - iou with ground truth (0 if iou < 0.5).

Then best separation threshold selected for each candidate according to predicted iou. Candidates with small predicted iou just removed (iou < 0.3 and iou < 0.2 for 2 submissions - it was hard to find this threshold using OOF prediction, because there is small overfit to image types even with such hard augmentation).

## 추가 데이터

 janowczyk, nucleisegmentationbenchmark, isbi2009, BBBC020, TNBC datasets에서 몇몇 이미지를 추가하였다.
 
그들 중 일부는 컬러 이미지에 대한 예측 정확도를 떨어뜨리면서 public LB를 감소시켰다. 

불행하게도 그 원인을 찾을 시간이 없어서 그대로 두었다. 

Additional data:

We added some images from janowczyk, nucleisegmentationbenchmark, isbi2009, BBBC020, TNBC datasets. Some of them decreased our score on Public LB, making less confident predictions on color images. Unfortunately we did not have time to find the culprits and left it as is. 

##Ensembling

후처리 이전에 마스크의 평균을 내는 간단한 앙상블 방법을 썼다. 



We used a simple approach for ensembling where we just averaged our masks before postprocessing

#Training


*   Random Crops: 256x256
*   Batch Size: 16
*   optimizer: Adam
*   learning rate : 초기값 1e-4에서 decay(여러 LR policy가 있지만  대부분 1e-4보다 작은 LR를 사용)
*   preprocessing : network에 따라 ImageNet과 동일

Training

    Random Crops: 256x256

    Batch Size: 16

    optimizer: Adam

    learning rate: initial 1e-4 with decay (we had different LR policies, but mostly small LR no more than 1e-4)

    preprocessing: same as on ImageNet depending on network


## Loss function

Sigmoid activation과 2채널 마스크가 있는 network의 경우 각 채널당 binary_crossentropy와 soft_dice를 조합하여 사용했습니다.

소프트맥스 활성화함수의 network 및 3 채널 마스크의 경우 범주형 crossentropy와 각 채널당 soft_dice를 조합하여 사용했습니다. (soft_dice는 마스크와 border channel에만 사용됨)

## Test Time Augmentation
Standard Flips/Rotations(0,90,180,270). 

## GitHub repo

https://github.com/selimsef/dsb2018_topcoders/

Loss function

For networks with sigmoid activation and 2 channel masks we used combination of binary_crossentropy with soft_dice per channel. For networks with softmax activation and 3 channel masks we used combination of categorical_crossentropy with soft_dice per channel (soft dice was applied only to mask and border channels).

Test Time Augmentations (TTA)

Standard Flips/Rotations(0,90,180,270). 

GitHub repo

https://github.com/selimsef/dsb2018_topcoders/